In [4]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import re

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show all columns in pandas
pd.set_option('display.max_columns', 500) 

# Graphing style
plt.style.use('seaborn-colorblind')

%matplotlib inline

In [7]:
urls = ["https://www.fantasypros.com/nfl/stats/qb.php?scoring=PPR",
        "https://www.fantasypros.com/nfl/stats/rb.php?scoring=PPR",
        "https://www.fantasypros.com/nfl/stats/wr.php?scoring=PPR",
        "https://www.fantasypros.com/nfl/stats/te.php?scoring=PPR",
        "https://www.fantasypros.com/nfl/stats/k.php?scoring=PPR",
        "https://www.fantasypros.com/nfl/stats/dst.php?scoring=PPR",
        "https://www.fantasypros.com/nfl/stats/db.php?scoring=PPR"]

positions = [x.split('/')[5].split('.')[0].upper() for x in urls]

dataframes = []

for url in urls: 
        # The read_html function returns a list of all the tables found on the web page
        tables = pd.read_html(url)

        # Assuming the table you want is the first one in the list, you can access it like this
        table_df = tables[0]

        position = url.split('/')[5].split('.')[0].upper()

        # Drop the first level of the MultiIndex, keeping only the second level
        if position in ['QB', 'RB', 'WR', 'TE']:
                table_df.columns = table_df.columns.droplevel(level=0)


        table_df.columns = [x.lower() for x in table_df.columns]

        table_df['pos'] = position

        # Extract team
        table_df['team'] = table_df['player'].apply(lambda x: x.split('(')[1].replace(')', '') if '(' in x else None)

        # Remove team from name
        table_df['player'] = table_df['player'].replace('\s\(.+\)', '', regex=True)

        dataframes.append(table_df)


In [8]:
df = dataframes[0]
df

,rank,player,cmp,att,pct,yds,y/a,td,int,sacks,att,yds,td,fl,g,fpts,fpts/g,rost,pos,team
0,1,Patrick Mahomes II,435,648,67.1,5250,8.1,41,12,26,61,358,4,0,17,429.4,25.3,100.0%,QB,KC
1,2,Josh Allen,362,573,63.2,4316,7.5,35,14,33,126,776,7,5,17,412.4,24.3,100.0%,QB,BUF
2,3,Jalen Hurts,306,460,66.5,3701,8.0,22,6,38,165,760,13,2,15,384.1,25.6,99.9%,QB,PHI
3,4,Joe Burrow,418,610,68.5,4527,7.4,36,12,41,76,259,5,3,17,369.0,21.7,99.9%,QB,CIN
4,5,Geno Smith,399,572,69.8,4282,7.5,30,11,46,68,366,1,4,17,314.9,18.5,88.1%,QB,SEA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,131,Jake Browning,0,0,0.0,0,0.0,0,0,0,0,0,0,0,0,0.0,0.0,0.0%,QB,CIN
131,132,C.J. Beathard,7,11,63.6,35,3.2,0,1,1,4,-4,0,0,4,0.0,0.0,0.1%,QB,JAC
132,133,Nate Sudfeld,0,0,0.0,0,0.0,0,0,1,5,-4,0,0,2,-0.4,-0.2,0.0%,QB,DET
133,134,Chad Henne,0,2,0.0,0,0.0,0,0,0,5,-5,0,0,3,-0.5,-0.2,0.1%,QB,FA
